In [74]:
# Try to decode which feature was selected per-trial based on firing rates of neurons
# experiment with ranges of firing rates around fixation (selection) time

%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats
from lfp_tools import (
    general as lfp_general,
    startup as lfp_startup,
    development as lfp_development,
    analysis as lfp_analysis
)
from spike_tools import (
    general as spike_general,
    analysis as spike_analysis,
)
import s3fs
import utils.behavioral_utils as behavioral_utils
import utils.spike_utils as spike_utils
import utils.classifier_utils as classifier_utils
from trial_splitters.random_splitter import RandomSplitter
from trial_splitters.block_splitter import BlockSplitter
from sklearn import svm

species = 'nhp'
subject = 'SA'
exp = 'WCST'
session = 20180802  # this is the session for which there are spikes at the moment. 

In [83]:
# grab behavioral data, spike data, trial numbers. 

fs = s3fs.S3FileSystem()

behavior_file = spike_general.get_behavior_path(subject, session)
behavior_data = pd.read_csv(fs.open(behavior_file))
valid_beh = behavior_data[behavior_data.Response.isin(["Correct", "Incorrect"])]
trial_numbers = np.unique(valid_beh.TrialNumber)
spike_data = spike_general.get_spike_times_by_trial(fs, subject, session, trial=trial_numbers, start_field="TrialStart", end_field="TrialEnd")    


In [5]:
# find "fixation times". In this case, 800ms before feedback. 
pre_interval = 800
post_interval = 0
intervals = behavioral_utils.get_trial_intervals(valid_beh, "FeedbackOnset", pre_interval, post_interval)

In [ ]:
# find the spikes falling in the intervals
spike_times = spike_general.get_spike_times(fs, subject, session)
spike_by_trial_interval = spike_utils.get_spikes_by_trial_interval(spike_times, intervals)

In [32]:
# grab firing rates, spike counts, in 800ms intervals, 100ms bins
firing_rates = spike_analysis.firing_rate(spike_by_trial_interval, bins=np.arange(0, 0.9, 0.1), smoothing=1)

In [88]:
# grab Shape, Color, Pattern features for each selection
selections = behavioral_utils.get_selection_features(valid_beh)

In [3]:
# load what's already been computed
feature_selections = pd.read_pickle("../data/feature_selections.pickle")
firing_rates = pd.read_pickle("../data/firing_rates.pickle")

In [62]:
# use a support vector machine model
clf = svm.SVC(decision_function_shape='ovo')

# Look at Spike Counts instead of Firing Rate
mode = "SpikeCounts"

# Look at 'Pattern' as a dimension
feature_dim = "Pattern"

# prep data for classification
inputs = firing_rates.rename(columns={mode: "Value"})
labels = feature_selections.rename(columns={feature_dim: "Feature"})


In [64]:
# randomly grab trials for train/test, run 20 times, split 80/20
random_splitter = RandomSplitter(labels.TrialNumber.unique(), 20, 0.2)
train_accs, test_accs, shuffled_accs, models = classifier_utils.evaluate_classifier(clf, inputs, labels, random_splitter)
print(f"Mean Train accuracy: {np.mean(train_accs)}, Test accuracy: {np.mean(test_accs)}, Shuffled accuracy: {np.mean(shuffled_accs)}")


In [ ]:
# split trials into train/test by blocks, always leave one block out for test. 
block_splitter = BlockSplitter(valid_beh)
train_accs, test_accs, shuffled_accs, models = classifier_utils.evaluate_classifier(clf, inputs, labels, block_splitter)

In [ ]:
# look across different time bins, evaluate classifier on each time bin individually
time_bins = np.arange(0.0, 0.8, 0.1)
for bin in time_bins:
    inputs_for_bin = inputs[inputs["TimeBins"] == bin]
    train_accs, test_accs, shuffled_accs, models = classifier_utils.evaluate_classifier(clf, inputs_for_bin, labels, random_splitter)
    print(f"Mean Train accuracy: {np.mean(train_accs)}, Test accuracy: {np.mean(test_accs)}, Shuffled accuracy: {np.mean(shuffled_accs)}")